In [66]:
from __future__ import print_function, division
import math
# from pandas import *
from scipy import stats
from numpy import *
import scipy.io as scio
# from sklearn.cross_validation import *
import numpy as np
import pandas as pd
import torch
# data read and prepare
# provide one hot encoding for discrete value
def encoding_(dict : dict, idx_to_stuff,stuff, ty=float) :
    num = len(dict.items())
    if dict.__contains__(stuff) : # 包含这个数值 则不进行处理 返回code
        return ty(dict[stuff])
    else : 
        dict[stuff] = num
        idx_to_stuff[num] = stuff
        return ty(num)

def encode_raw_data(raw_data_set : pd.DataFrame, di_raw= None) :
    out_matrix = np.zeros(raw_data_set.shape)
    # size = raw_data_set.items()
    # print(size)
    ret = []
    for ind in range(raw_data_set.shape[1]) :
        col = raw_data_set[ind]
        dic = dict()
        idx_dic = dict()
        if di_raw:
            dic,idx_dic = di_raw[ind] 
        idx = 0
        for elem in col : 
            elem_code  = encoding_(dic, idx_dic, elem)
            out_matrix[idx,ind] = elem_code
            idx += 1
        ret.append((dic, idx_dic))
    return out_matrix, ret

In [67]:
raw_data_test = pd.read_csv("C:\\Users\\coco1\\PycharmProjects\\PRHM01\\03\\ds03\\adult.test", header=None, dtype=str)
raw_data = pd.read_csv("C:\\Users\\coco1\\PycharmProjects\\PRHM01\\03\\ds03\\adult.data", header=None, dtype=str)
train,di_raw = encode_raw_data(raw_data)
test,di_raw = encode_raw_data(raw_data_test, di_raw)

In [68]:
train_set = torch.tensor(train[:,0:-1], dtype=torch.float)
train_label = torch.tensor(train[:,-1], dtype=torch.int)
test_set = torch.tensor(test[:,0:-1], dtype=torch.float)
test_label = torch.tensor(test[:,-1], dtype=torch.int)

In [78]:
def modify_covariance(cov):
    a, b = np.linalg.eig(cov)
    a_modeified = [0 for i in range(len(a))]
    sum_a = sum(a)
    c = 0
    for i in range(len(a)):
        index_i = np.where(a==sorted(a,reverse=True)[i])[0][0]
        a_modeified[index_i] = a[index_i]
        c += a[index_i]
        cc = float(c)/sum_a
        delta = float(sum_a-c)/float(len(a) - index_i + 1)
        while 0.99 <= cc:
            a_modeified[index_i] = delta
            index_i +=1
            if len(a) == index_i:
                break
    cov_modefied = np.dot(np.dot(np.linalg.inv(b), mat(diag(a))),b)
    return cov_modefied

def get_qda_cor_matrix (train, mu) :
    print(len(train))
    size = train[0].shape[0]
    zero: np.ndarray = np.random.normal(0, 0.01, size=(size,size))
    i = 1
    for vec in train : 
        if i == 1 :
            print(type(vec - mu))
            print(type(zero))
            i += 1
        zero += np.multiply((vec - mu), vec - mu)
    
    return zero / len(train)

def get_qda_norm_attribute (train) : 
    init_vec = np.zeros(train[0].shape[0])
    for vec in train :
        init_vec += vec
    return init_vec / len(train)

def get_sample_covariance_matrix(sample_data):
    mean_i = get_qda_norm_attribute(sample_data)
    sample_data_1 = sample_data
        #print type(sample_data_1)
    covMatrix = get_qda_cor_matrix(sample_data_1, mean_i)
        #covMatrix = np.cov(sample_data)
    return covMatrix,mean_i

In [79]:
type1_train_data = []
type2_train_data = []
num_1 = 0
for i in range(train_label.shape[0]) :
    if train_label[i] == 0 : 
        type1_train_data.append(train_set[i])
        num_1 += 1
    if train_label[i] == 1 : type2_train_data.append(train_set[i])
    # print(str(train_label[i]))

In [80]:
covMatrix_1, mean_1 = get_sample_covariance_matrix(
    np.array([[float(elem) 
               for elem in arr]for arr in type1_train_data]))
modify_covmatrix_1 = modify_covariance(covMatrix_1)

covMatrix_2, mean_2 = get_sample_covariance_matrix(
    np.array([[float(elem) 
               for elem in arr]for arr in type2_train_data]))
modify_covmatrix_2 = modify_covariance(covMatrix_2)

24720
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
7841
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: ComplexWarning: Casting complex values to real discards the imaginary part
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: ComplexWarning: Casting complex values to real discards the imaginary part
  # This is added back by InteractiveShellApp.init_path()


In [81]:
p_1 = num_1 / train_label.shape[0]
p_2 = 1- p_1

In [112]:
def discriminant_function(x, covMatrix, mean_i,p_i):
        cov_inverse = np.linalg.pinv(covMatrix)
        a = x - mean_i
        b = np.dot(a.T, cov_inverse)
        c = -np.dot(b, a)
        discriminant_function_value = c - math.log(p_i)
        return discriminant_function_value

In [114]:
ind = 0
right = 0
for elem in test_set : 
    elem = elem.numpy()
    r_1 = discriminant_function(elem, modify_covmatrix_1, mean_1, p_1)
    # break
    r_2 = discriminant_function(elem, modify_covmatrix_2, mean_2, p_2)
    num = 2
    # print(r_1)
    if r_1 < r_2: num += 0
    else : num += 1
    if int(test_label[ind]) == num : right += 1 
    ind += 1
    # break
    
print(right / test_label.shape[0])

0.7567716970702045


In [115]:
int(test_label[0]) 

2